# Conversión de  datos a formato NILMTK y carga en NILMTK

NILMTK utiliza un formato de archivo abierto basado en el formato de archivo binario HDF5 para almacenar tanto los datos de energía como los metadatos. El primer paso al usar NILMTK es convertir su conjunto de datos al formato de archivo NILMTK HDF5

**NOTA**: Si está en Windows, recuerde evitar las barras diagonales inversas, usar barras inclinadas hacia adelante o usar cadenas sin formato al pasar rutas en Python, p. Ej. uno de los siguientes funcionaría:

```python
convert_redd('c:\\data\\REDD\\low_freq', r'c:\\data\\redd.h5')
convert_redd('c:/data/REDD/low_freq', 'c:/data/redd.h5')
convert_redd(r'c:\data\REDD\low_freq', r'c:\data\redd.h5')
```

## DSUALM

Convertir el conjunto de datos DSUALM es fácil

In [1]:
from nilmtk.dataset_converters import convert_ualm
convert_ualm('./data/ozm/', 'dsual.h5')

Path ualm: ./data/ozm/ /electricity
..Cargando   1
Fichero  1 cargado ok
..Cargando   2
Fichero  2 cargado ok
..Cargando   3
Fichero  3 cargado ok
..Cargando   4
Fichero  4 cargado ok
..Cargando   5
Fichero  5 cargado ok
..Cargando   6
Fichero  6 cargado ok
..Cargando   7
Fichero  7 cargado ok
Uniendo  Medadata
Done converting YAML metadata to HDF5!
Realizada con exito la conversion de ualM al formato  HDF5!


Ahora redd.h5 contiene todos los datos de energía de REDD y todos los metadatos relevantes. En NILMTK v0.2, esta conversión solo usa una pequeña fracción de la memoria del sistema (a diferencia de NILMTK v0.1, que consumiría ~ 1 GByte de RAM solo para realizar la conversión del conjunto de datos).

Por supuesto, si desea ejecutar convert_redd en su propia máquina, primero debe descargar [REDD](http://redd.csail.mit.edu), descomprimirlo y pasar el source_directory y el output_filename relevantes a convert_redd 

## Otros datasets

En el momento de escribir, [NILMTK contains converters for 8 datasets](https://github.com/nilmtk/nilmtk/tree/master/nilmtk/dataset_converters).

¡Contribuir con un nuevo convertidor es fácil y muy recomendable!  [Learn how to write a dataset converter](https://github.com/nilmtk/nilmtk/blob/master/docs/manual/development_guide/writing_a_dataset_converter.md).

## Abrir  HDF5 en NILMTK

In [2]:
from nilmtk import DataSet
from nilmtk.utils import print_dict

#dataset de referencia REDD
redd = DataSet('dsual.h5')

En este punto, todos los metadatos se han cargado en la memoria, pero no se ha cargado ninguno de los datos de energía. Este es nuestro primer encuentro con una diferencia fundamental entre NILMTK v0.1 y v0.2 +: NILMTK v0.1 solía cargar con entusiasmo todo el conjunto de datos en la memoria antes de que usted hiciera ningún trabajo real en los datos. ¡NILMTK v0.2 + es perezoso! No cargará datos en la memoria hasta que le diga lo que quiere hacer con los datos (e, incluso entonces, se cargarán grandes conjuntos de datos en fragmentos que quepan en la memoria). Esto permite que NILMTK v0.2 + funcione con conjuntos de datos arbitrariamente grandes (conjuntos de datos demasiado grandes para caber en la memoria) sin obstruir su sistema.

### Explorando el `DataSet` object

Echemos un vistazo rápido para ver qué hay en este objeto `redd` ...

Hay muchos metadatos asociados con el conjunto de datos, incluida información sobre los dos modelos de dispositivo de medición que los autores utilizaron para registrar REDD:

In [3]:
print_dict(redd.metadata)

También tenemos todos los edificios disponibles como [OrderedDict] (https://docs.python.org/2/library/collections.html#collections.OrderedDict) (indexados desde 1 y no 0 porque cada conjunto de datos que conocemos comienza a numerar edificios de 1 no 0

In [4]:
print_dict(redd.buildings)

Cada edificio tiene un poco de metadatos asociados (no hay muchos metadatos específicos del edificio en REDD):

In [5]:
print_dict(redd.buildings[1].metadata)

Cada edificio tiene un atributo elec que es un objeto MeterGroup (¡mucho más sobre ellos pronto!)

In [6]:
redd.buildings[1].elec

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='DSUAL', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='DSUAL', appliances=[Appliance(type='boiler', instance=1)])
  ElecMeter(instance=3, building=1, dataset='DSUAL', appliances=[Appliance(type='fan', instance=1)])
  ElecMeter(instance=4, building=1, dataset='DSUAL', appliances=[Appliance(type='freezer', instance=1)])
  ElecMeter(instance=5, building=1, dataset='DSUAL', appliances=[Appliance(type='television', instance=1)])
  ElecMeter(instance=6, building=1, dataset='DSUAL', appliances=[Appliance(type='vacuum cleaner', instance=1)])
  ElecMeter(instance=7, building=1, dataset='DSUAL', appliances=[Appliance(type='light', instance=1)])
)

¡Sí, asi es , es aqui donde estan los datos importantes!